In [1]:
import pyarrow.dataset as ds

# dataset = ds.dataset("/home/doserbd/data/gaia/xp_sampled_mean_spectrum/parquet/XpSampledMeanSpectrum_000000-003111.parquet",
dataset = ds.dataset("../tests/data/XpSampledMeanSpectrum", format="parquet")
print(dataset.to_table().schema.metadata[b"flux_shape"])

b'(1, 344)'


In [2]:
from hipster import CatalogGenerator, Inference

catalog_generator = CatalogGenerator(
    encoder=Inference("../tests/models/vae_encoder.onnx"),
    data_directory="../tests/data/XpSampledMeanSpectrum",
    url="http://localhost:8083",
    title="gaia",
)

catalog = catalog_generator()

2025-01-26 23:20:14.951365389 [W:onnxruntime:, graph.cc:109 MergeShapeInfo] Error merging shape info for output. 'encoder_1' source:{} target:{1}. Falling back to lenient merge.


In [3]:
from astropy.io.votable import writeto
from astropy.table import Table

table = Table.from_pandas(catalog)
writeto(table, "catalog.vot")

In [4]:
import numpy as np
flux = np.array([[[1, 2, 3], [4, 5, 6]], [[7, 8, 9], [10, 11, 12]]], dtype=np.float32)

for i, x in enumerate(flux):
    flux[i] = (x - x.min()) / (x.max() - x.min())

flux

array([[[0. , 0.2, 0.4],
        [0.6, 0.8, 1. ]],

       [[0. , 0.2, 0.4],
        [0.6, 0.8, 1. ]]], dtype=float32)

In [5]:
from operator import le
from matplotlib.pylab import f
from hipster import ImageGenerator, Inference

ImageGenerator(
    encoder=Inference("../tests/models/vae_encoder.onnx"),
    decoder=Inference("../tests/models/vae_decoder.onnx"),
    data_directory="../tests/data/XpSampledMeanSpectrum",
    batch_size=2,
    figsize_in_pixel=(120, 90),
    legend=False,
)()

2025-01-26 23:20:15.033429291 [W:onnxruntime:, graph.cc:109 MergeShapeInfo] Error merging shape info for output. 'encoder_1' source:{} target:{1}. Falling back to lenient merge.
